# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Import API key
from api_keys import g_key

file = "output_data/cities.csv" 
weather = pd.read_csv(file) 

weather = pd.read_csv(file) 
weather.set_index('City Name', inplace=True) 
weather.drop(columns=['Unnamed: 0'], inplace=True) 
weather.reset_index()



,City Name,Date,Long,Lat,Temp,Humid,Wind,clouds,Country
0,Vaini,1612023597,-175.2000,-21.2000,73.40,94,4.61,90,TO
1,Pangnirtung,1612023197,-65.7125,66.1451,-9.40,77,2.26,1,CA
2,Yellowknife,1612023218,-114.3525,62.4560,-18.99,76,5.75,20,CA
3,Albany,1612023161,-73.9662,42.6001,14.00,53,3.00,0,US
4,Ankang,1612023598,109.0172,32.6800,49.06,48,0.85,99,CN
...,...,...,...,...,...,...,...,...,...
554,Itupiranga,1612023458,-49.3267,-5.1347,89.60,66,4.61,40,BR
555,Morondava,1612023462,44.2833,-20.2833,83.08,74,5.84,1,MG
556,Omboué,1612023404,9.2618,-1.5746,81.63,76,10.47,44,GA
557,Sitka,1612023393,-135.3300,57.0531,37.40,60,9.22,75,US


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations = weather[["Lat", "Long"]].astype(float)
weight = weather['Humid']
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2.5)

fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

In [4]:
hotel_df = weather[weather.eval('Temp>=65 & Temp < 90 & (Humid < 40) & clouds <= 50')]
hotel_df.reset_index(inplace=True)
hotel_df


,City Name,Date,Long,Lat,Temp,Humid,Wind,clouds,Country
0,Adrar,1612023637,-10.0711,20.5022,85.51,12,12.91,25,MR
1,Marsá Maţrūḩ,1612023583,27.2453,31.3525,73.40,35,20.71,17,EG
2,Umm Kaddadah,1612023647,26.6876,13.6017,74.01,18,14.36,0,SD
3,Cosalá,1612023652,-106.6908,24.4125,78.93,35,1.54,0,MX
4,Constantine,1612023667,6.6147,36.3650,69.80,32,14.97,0,DZ
5,São José dos Campos,1612023670,-45.8869,-23.1794,89.60,33,3.44,40,BR
6,Minas Novas,1612023680,-42.5903,-17.2186,88.63,36,6.13,8,BR
7,Doka,1612023402,35.7667,13.5167,85.17,23,10.83,0,SD
8,Zuwārah,1612023689,12.0820,32.9312,72.86,36,10.83,0,LY
9,Jalu,1612023698,21.5482,29.0331,70.20,30,9.66,0,LY


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
hotel_df["Hotel"] = ""
hotel_df.head()

<ipython-input-5-77cb39ae4dbd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel"] = ""


,City Name,Date,Long,Lat,Temp,Humid,Wind,clouds,Country,Hotel
0,Adrar,1612023637,-10.0711,20.5022,85.51,12,12.91,25,MR,
1,Marsá Maţrūḩ,1612023583,27.2453,31.3525,73.40,35,20.71,17,EG,
2,Umm Kaddadah,1612023647,26.6876,13.6017,74.01,18,14.36,0,SD,
3,Cosalá,1612023652,-106.6908,24.4125,78.93,35,1.54,0,MX,
4,Constantine,1612023667,6.6147,36.3650,69.80,32,14.97,0,DZ,


In [7]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
target_search = "hotel"
target_radius = 5000
target_type = "lodging"

# set up a parameters dictionary
params = {
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat = row['Lat']
    long = row['Long']
    city = row['City Name']
    #append the lat and long from each row and add to 
    params["location"] = f'{lat},{long}'

    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    try:
        hotel_df.loc[index, "Hotel"] = results[0]['name']
    except :
        pass
    time.sleep(1)
    print("------------")

    hotel_df.dropna(how='any', inplace=True)
    hotel_df

Retrieving Results for Index 0: Adrar.
------------
Retrieving Results for Index 1: Marsá Maţrūḩ.


<ipython-input-7-1856078e8dc1>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df.dropna(how='any', inplace=True)
/Users/joyitaroy/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


------------
Retrieving Results for Index 2: Umm Kaddadah.


<ipython-input-7-1856078e8dc1>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df.dropna(how='any', inplace=True)


------------
Retrieving Results for Index 3: Cosalá.


...

<ipython-input-7-1856078e8dc1>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df.dropna(how='any', inplace=True)


------------
Retrieving Results for Index 11: Araouane.


<ipython-input-7-1856078e8dc1>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df.dropna(how='any', inplace=True)


------------


<ipython-input-7-1856078e8dc1>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df.dropna(how='any', inplace=True)


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Long"]]

In [ ]:
#gmaps.configure(api_key=g_key)

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)


'''figure_layout = {
    'width': '450px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(layout=figure_layout)'''
fig.add_layer(markers)
# Display figure
fig


Figure(layout=FigureLayout(height='420px'))